## Quiz #0701

### "TensorFlow machine learning with Calilfornia housing data"

In [1]:
# Compatibility with tf 1.x
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
import matplotlib.pyplot as plt
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')    # Turn the warnings off.

Instructions for updating:
non-resource variables are not supported in the long term


#### Answer the following question by providing Python code:

In [2]:
# Bring the data.
housing_data = fetch_california_housing()

In [3]:
# Read the description.
print(housing_data['DESCR'])

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

1). Explore the data:
- Display the dataset as a DataFrame with column labels (data + labels).

2). Build a machine learning model with TensorFlow. 
- Preprocess the data if necessary.
- Build a linear **regression** model.
- Train the model.
- Calculate the error metrics such as MSE and RMSE (in-sample or training and out-of-sample or testing). Target: RMSE < 1.

In [4]:
#Primero vemos los datos cargados
print(housing_data.keys())
X = housing_data['data']                  # Get the data from the dataset ('data' attribute)
y = housing_data['target'] 
# Imprimimos las dimensiones de X e y
print(X.shape)
print(y.shape)

dict_keys(['data', 'target', 'feature_names', 'DESCR'])
(20640, 8)
(20640,)


In [5]:
y = y.reshape(-1,1) # Redimensionamos el vector y a una matriz de tamaño (20640,1), para no tener problemas con la operativa
print(X.shape)
print(y.shape)

(20640, 8)
(20640, 1)


In [6]:
new_data = np.concatenate((X,y), axis=1) #Unimos los arrays de X e y en un mismo plano para construir la variable sobre la que contruiremos el dataframe
#Definimos las columnas
#column_names = ['MedInc','HouseAge','AveRooms','AveBedrms','Population','AveOccup', 'Latitude', 'Longitude']
df = pd.DataFrame(data= new_data, columns= housing_data.feature_names + ['MedHouseVal']) #Añadimos un nombre para la columna que alberga la variable y
df # Mostramos el dataframe

# No necesitamos buscar valores nulos ya que en las características del dataset nos garantizan que no los hay.

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [7]:
print(np.min(X), np.max(X), np.mean(X), np.std(X))
X = scale(X)
print(np.min(X), np.max(X), np.mean(X), np.std(X))

-124.35 35682.0 172.9556735813799 621.6720986716564
-2.3859923416733877 119.41910318829312 -1.1890574657535784e-15 0.9999999999999999


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)  # Dividimos las variables 'X' e 'y' en dos secciones: "entrenamiento" (70%) y test (30%)
n_train_size = X_train.shape[0]
print(X_train.shape)                                    # Mostramos las dimensiones de X_train, X_test, y_train, y_test      
print(X_test.shape) 
print(y_train.shape) 
print(y_test.shape)

(14448, 8)
(6192, 8)
(14448, 1)
(6192, 1)


In [9]:
# Definimos los siguientes parámetros:
batch_size = 32                                # Tamaño de cada 'batch'.
n_epochs  = 50000                              # Nº de épocas.
learn_rate = 0.0001                            # Tasa de aprendizaje

In [10]:
# Definimos las Variables y Placeholders
W =  tf.Variable(tf.random_normal([8,1]), name='W', dtype=tf.float32)  # Define la matriz W (tipo tf.float32)
b =  tf.Variable(tf.random_normal([1]), name='b', dtype=tf.float32)  # Por cada valor de salida, se requiere un bias.

X_ph =  tf.placeholder(tf.float32, shape=(None,8), name='X_ph') # Define un placeholder de tipo float con un nº de filas no especificado.
y_ph =  tf.placeholder(tf.float32, shape=(None), name='y_ph')   # Define un placeholder de tipo float con nº de columnas igual al nº de variables de regresión.

In [11]:
# Definimos el modelo
y_model = tf.matmul(X_ph,W) + b  # Modelo de regresión linear simple. 

In [12]:
#Definimos la función de pérdida y el método de optimización
loss = tf.reduce_mean(tf.square(y_ph-y_model)) # Define el error cuadrático medio (L2 Error)
optimizer = tf.train.MomentumOptimizer(learning_rate=learn_rate, momentum=0.9) # Define el optimizador 'momentum' con una tasa de aprendizaje igual a la anteriormente definida y momentum = 0.9 (valor por defecto)
train = optimizer.minimize(loss)  # Define el entrenamiento en base al optimizador de pérdida.
init = tf.global_variables_initializer()  # Define la inicialización de las variables anteriores.

In [13]:
# Realizamos el proceso de entrenamiento. Al final, calcularemos el mse para el entrenamiento y es test.
with tf.Session() as sess:
    sess.run(init)
    # Fase de entrenamiento (realizando los pasos contenidos a partir del 'for' en sucesivos bucles definidos posteriormente en base a la épocas marcadas anteriormente).
    for i in range(n_epochs):
        idx_rnd = np.random.choice(range(n_train_size),batch_size,replace=False)  # Se generan los índices para la selección de las muestras o batch.                
        batch_X, batch_y = X_train[idx_rnd,:], y_train[idx_rnd,:] # Seleccionamos los batch y los entrenamos.
        # Metemos los datos con los que vamos a alimentar al modelo en un diccionario y se los pasamos                   
        my_feed = {X_ph:batch_X, y_ph:batch_y}
        sess.run(train, feed_dict = my_feed)
        
        if (i + 1) % 500 == 0:
            l = sess.run(loss, feed_dict = my_feed)
            print("Step : {}, Loss : {}".format(i + 1, l))
    # Fase de testing
    mse = loss
    # Tras definir el mse en función a la variable de pérdida, calculamos dicho error para los datos de entrenamiento y test
    mse_train = sess.run(mse, feed_dict = {X_ph:X_train, y_ph: y_train})
    mse_test = sess.run(mse, feed_dict = {X_ph:X_test, y_ph: y_test})

Step : 500, Loss : 2.2003204822540283
Step : 1000, Loss : 0.9563908576965332
Step : 1500, Loss : 0.9113023281097412
Step : 2000, Loss : 0.44624221324920654
Step : 2500, Loss : 1.0850332975387573
Step : 3000, Loss : 0.6153214573860168
Step : 3500, Loss : 0.6917641758918762
Step : 4000, Loss : 1.107374668121338
Step : 4500, Loss : 0.8523281812667847
Step : 5000, Loss : 1.0105079412460327
Step : 5500, Loss : 0.28934425115585327
Step : 6000, Loss : 0.6063458919525146
Step : 6500, Loss : 0.4203428328037262
Step : 7000, Loss : 0.43730711936950684
Step : 7500, Loss : 0.8171295523643494
Step : 8000, Loss : 0.5885061025619507
Step : 8500, Loss : 0.4655064642429352
Step : 9000, Loss : 0.6800311803817749
Step : 9500, Loss : 0.46337753534317017
Step : 10000, Loss : 0.5928782820701599
Step : 10500, Loss : 1.1964126825332642
Step : 11000, Loss : 0.3585911691188812
Step : 11500, Loss : 0.5250943899154663
Step : 12000, Loss : 0.4124101400375366
Step : 12500, Loss : 0.6995092034339905
Step : 13000, Los

In [16]:
# Mostramos el MSE y the RMSE (la raíz cuadrada del MSE) para los conjuntos de datos de entrenamiento y test.

print("Training: MSE - {}, RMSE - {}".format(mse_train + 1, np.sqrt(mse_train)))
print("Test: MSE - {}, RMSE - {}".format(mse_train + 1, np.sqrt(mse_train)))

Training: MSE - 1.52139812707901, RMSE - 0.7220790386199951
Test: MSE - 1.52139812707901, RMSE - 0.7220790386199951


Obtenemos un resultado del RMSE menor que 1, como era requerido.